In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Cargar el dataset principal con los datos meteorológicos de las estaciones
df_meteorologico = pd.read_excel('./Datos/REMAS_Etchojoa_Cajeme_2022-2023.xlsx')

In [5]:
# Cargar el dataset con los valores de ETo
df_eto = pd.read_excel('./Datos/ETo_REMAS_Etchojoa_Cajeme_2022-2023.xlsx')

In [ ]:
# Filtrar por el municipio y la estación deseada
municipio_seleccionado = "Cajeme"  # Cambia el municipio aquí
estacion_seleccionada = "Estación_Ejemplo"  # Cambia el nombre de la estación específica aquí

df_meteorologico = df_meteorologico[
    (df_meteorologico['Municipio'] == municipio_seleccionado) & 
    (df_meteorologico['Estación'] == estacion_seleccionada)
]

In [5]:
# Renombrar las columnas en df_eto para facilitar la unión
df_eto.rename(columns={'DDR': 'Estación', 'FECHA': 'Fecha'}, inplace=True)

# Asegurarnos de que ambas columnas de fecha están en el mismo formato
df_meteorologico['Fecha'] = pd.to_datetime(df_meteorologico['Fecha'], format='%m/%d/%Y')
df_eto['Fecha'] = pd.to_datetime(df_eto['Fecha'], format='%m/%d/%Y')

# Filtrar el DataFrame de ETo por la misma estación
df_eto = df_eto[df_eto['Estación'] == estacion_seleccionada]

# Unir ambos DataFrames en función de la estación (Estación) y la fecha (Fecha)
df = pd.merge(df_meteorologico, df_eto[['Estación', 'Fecha', 'ETO']], on=['Estación', 'Fecha'], how='inner')

# Definir los parámetros del cultivo (trigo)
pcc = 140  # Periodo de crecimiento del cultivo en días
kc_values = {
    'inicio': 0.15,  # Kc inicial
    'medio': 1.15,   # Kc medio
    'final': 0.50    # Kc final
}
rendimiento = 5.0  # Ejemplo de rendimiento del cultivo en ton/ha

# Definir la fecha de inicio en el código
fecha_inicio_str = "11/30/2022"  # Cambia la fecha de inicio según el periodo deseado
fecha_inicio = datetime.strptime(fecha_inicio_str, '%m/%d/%Y')

# Verificar si hay suficientes días para el periodo de cultivo
fecha_fin = fecha_inicio + timedelta(days=pcc - 1)
ultima_fecha = df['Fecha'].max()

if fecha_fin > ultima_fecha:
    print(f"No hay suficientes datos para un periodo de {pcc} días desde la fecha de inicio {fecha_inicio_str}.")
else:
    # Filtrar el DataFrame para el periodo de 140 días desde la fecha de inicio y hacer una copia para evitar warnings
    df_periodo = df[(df['Fecha'] >= fecha_inicio) & (df['Fecha'] <= fecha_fin)].copy()

    # Crear y asignar la columna 'Kc' como float para evitar problemas de tipo de datos
    df_periodo['Kc'] = 0.0
    df_periodo.loc[df_periodo.index < 45, 'Kc'] = kc_values['inicio']
    df_periodo.loc[(df_periodo.index >= 45) & (df_periodo.index < 95), 'Kc'] = kc_values['medio']
    df_periodo.loc[df_periodo.index >= 95, 'Kc'] = kc_values['final']

    # Calcular ETc (Evapotranspiración de cultivo)
    df_periodo['ETc'] = df_periodo['Kc'] * df_periodo['ETO']

    # Definir la función para calcular Precipitación efectiva
    def calcular_precip_efectiva(precipitacion_total):
        if precipitacion_total < 250:
            # Cálculo de la precipitación efectiva cuando es menor a 250 mm
            precip_efectiva = precipitacion_total * (125 - 0.2 * precipitacion_total) / 125
        else:
            # Cálculo de la precipitación efectiva cuando es mayor o igual a 250 mm
            precip_efectiva = 125 + 0.1 * precipitacion_total
        return precip_efectiva

    # Calcular Precipitación efectiva para cada registro
    df_periodo['Precipitación efectiva'] = df_periodo['Precipitación (mm)'].apply(calcular_precip_efectiva)

    # Calcular ETverde (Evapotranspiración de agua verde)
    df_periodo['ETverde'] = df_periodo[['ETc', 'Precipitación efectiva']].min(axis=1)

    # Calcular ETazul (Evapotranspiración de agua azul)
    df_periodo['ETazul'] = (df_periodo['ETc'] - df_periodo['Precipitación efectiva']).apply(lambda x: max(0, x))

    # Calcular el uso de agua verde y agua azul total en el periodo de 140 días
    uso_agua_verde = 10 * df_periodo['ETverde'].sum()  # m³/ha
    uso_agua_azul = 10 * df_periodo['ETazul'].sum()    # m³/ha

    # Calcular la Huella Hídrica en m³/ton para agua verde y agua azul
    hh_proc_verde = uso_agua_verde / rendimiento
    hh_proc_azul = uso_agua_azul / rendimiento

    # Mostrar los resultados
    print(f"Uso de Agua Verde en Cultivo para los {pcc} días desde {fecha_inicio_str}: {uso_agua_verde} m³/ha")
    print(f"Uso de Agua Azul en Cultivo para los {pcc} días desde {fecha_inicio_str}: {uso_agua_azul} m³/ha")
    print(f"Huella Hídrica de Agua Verde (HHproc, verde): {hh_proc_verde} m³/ton")
    print(f"Huella Hídrica de Agua Azul (HHproc, azul): {hh_proc_azul} m³/ton")

Uso de Agua Verde en Cultivo para los 140 días desde 11/30/2022: 56.3126 m³/ha
Uso de Agua Azul en Cultivo para los 140 días desde 11/30/2022: 3360.9673999999995 m³/ha
Huella Hídrica de Agua Verde (HHproc, verde): 11.26252 m³/ton
Huella Hídrica de Agua Azul (HHproc, azul): 672.1934799999999 m³/ton
